In [7]:
from collections import defaultdict
from PIL import Image
from pathlib import Path
import imagehash
from tqdm import tqdm
import shutil
from typing import List

In [9]:
def find_similar_images(
    image_paths: List[Path],
    threshold: int = 5
) -> dict[str, List[Path]]:
    """
    Группирует похожие изображения на основе хеша.
    
    :param image_paths: список путей к изображениям
    :param threshold: максимальное расстояние между хешами (0-64)
                     чем меньше, тем строже сравнение
    :return: {representative_hash: [список похожих путей]}
    """
    hashes = {}
    groups = defaultdict(list)
    
    # Сначала собираем все хеши
    for path in tqdm(image_paths, desc="Compute hashes"):
        try:
            img = Image.open(path)
            h = imagehash.phash(img)
            hashes[path] = h
        except Exception as e:
            print(f"Ошибка обработки {path}: {e}")
    
    # Группируем похожие изображения
    used_paths = set()
    for path1, hash1 in tqdm(hashes.items(), desc="Grouping similar images"):
        if path1 in used_paths:
            continue
            
        current_group = [path1]
        for path2, hash2 in hashes.items():
            if path2 not in used_paths and path1 != path2:
                if hash1 - hash2 <= threshold:  # Расстояние Хемминга
                    current_group.append(path2)
        
        if len(current_group) > 1:  # Только группы с похожими изображениями
            for p in current_group:
                used_paths.add(p)
            # Используем hex-представление хэша как ключ
            groups[str(hash1)].extend(current_group)
    
    return dict(groups)

In [10]:
def find_and_move_duplicates(
    image_paths: List[Path],
    output_dir: Path,
    threshold: int = 5,
    min_group_size: int = 2
) -> int:
    """
    Находит дубликаты в папке и перемещает их в подпапки output_dir.
    
    :param input_dir: Папка с изображениями для проверки
    :param output_dir: Папка для сохранения групп дубликатов
    :param threshold: Максимальное расстояние между хешами (0-64)
    :param min_group_size: Минимальный размер группы для перемещения
    """
    if not output_dir.exists():
        output_dir.mkdir(parents=True)
    
    # Находим похожие изображения
    groups = find_similar_images(image_paths=image_paths, threshold=threshold)
    
    # Перемещаем группы дубликатов
    moved = 0
    for group_num, (hash_val, paths) in enumerate(groups.items(), 1):
        if len(paths) >= min_group_size:
            # Первый файл в группе считаем оригиналом (не перемещаем)
            original = paths[0]
            filename_without_suffix = original.name.replace(original.suffix, '')
            group_dir = output_dir / f"duplicates_{filename_without_suffix}_{group_num}"
            group_dir.mkdir(exist_ok=True)
            
            # Перемещаем дубликаты
            for duplicate in paths[1:]:
                shutil.move(duplicate, group_dir / duplicate.name)
                # print(f"Дубликат → {group_dir.name}/{duplicate.name}")
                moved += 1
    
    return moved

In [43]:
current_dir = Path.cwd()
print(f"Директория: {current_dir}")
data_dir = current_dir.parent.parent / "data"
print(f"data_dir: {data_dir}")

dataset_dir = data_dir / "interior_dataset"
all_dataset_images = []
for path in sorted(dataset_dir.iterdir()):
    if path.is_dir():
        image_paths = sorted(path.iterdir())
        all_dataset_images.extend(image_paths)

moved = find_and_move_duplicates(image_paths=all_dataset_images, output_dir=dataset_dir / "dublicates")
print(f"{path.name} | Moved {moved}")


Директория: /home/little-garden/CodeProjects/InteriorClass/src/notebooks
data_dir: /home/little-garden/CodeProjects/InteriorClass/data


Grouping similar images: 100%|██████████| 54617/54617 [1:22:30<00:00, 11.03it/s]

D1 | Moved 26
